In [1]:
import warnings; warnings.simplefilter('ignore')

In [2]:
from qiskit import QuantumCircuit, execute, Aer, IBMQ,QuantumRegister,ClassicalRegister
from qiskit.compiler import transpile, assemble
from qiskit.tools.monitor import job_monitor
from qiskit.tools.jupyter import *
from qiskit.visualization import *
import networkx as nx

import operator
from numpy import flip,array,binary_repr,insert

import math

$$\newcommand{\ket}[1]{\left|{#1}\right\rangle}$$
$$\newcommand{\bra}[1]{\left\langle{#1}\right|}$$

## Configuration

### Graph coloring problem definition

The defined parameter `NO_OF_NODES` represent the number of nodes in graph. The parameter is passed as parameter to `init_graph` function to generate an Erdos-Renyi graph with the configured number of nodes.

In [3]:
NO_OF_NODES = 5

In [4]:
def init_graph(no_of_nodes,edge_probability):
    # graph = nx.fast_gnp_random_graph(
    #     no_of_nodes,
    #     edge_probability,seed=None,directed=False)
    #graph = nx.erdos_renyi_graph(no_of_nodes,edge_probability,seed=None,directed=False)
    #adjacency_matrix=nx.to_numpy_matrix(graph).astype(int)
    
    AdjMatrix = array([[0,1,0,0,1],[1,0,1,0,0],[0,1,0,1,0],[0,0,1,0,1],[1,0,0,1,0]])
    G = nx.from_numpy_matrix(AdjMatrix)

    graph = nx.from_numpy_matrix(AdjMatrix)
    adjacency_matrix=nx.to_numpy_matrix(graph).astype(int)
    return adjacency_matrix

In [5]:
GRAPH = init_graph(NO_OF_NODES,0.4).tolist()

## Algorithm configuration parameters

The following parameters are defined and used in the algorithm implementation:

* `NO_OF_COLORS` -- the maximum number of colors used for coloring the graph.
* `INVALID_COLORS_LIST` -- the list of invalid colors. Binary colors are specified as tuples ( e.g. `[(1,1),(0,1)]` for marking the color configurations `11` and `01` as invalid).
* `NO_OF_QUBITS_FITNESS` -- the number of qubits needed for representing the fitness value.
* `NO_OF_QUBITS_PER_COLOR` -- the number of qubits needed for representing a color configuration.
* `NO_OF_QUBIT_INDIVIDUAL` -- the total number of qubits needed for representing an individual. Thus, considering that an individual is made of chromosomes that represents the color configurations for each node, the total number of qubits is represented by the product between `NO_OF_NODES` and `NO_OF_QUBITS_PER_COLOR`.
* `POPULATION_SIZE` -- all possible configurations for coloring the graph
* `NO_OF_MAX_GROVER_ITERATIONS` -- the maximum number of Grover iterations in which a solution should be found.




In [6]:
NO_OF_COLORS = 3# 2 bits needed
INVALID_COLORS_LIST = [(1,1)]
NO_OF_QUBITS_FITNESS = 4
NO_OF_QUBITS_PER_COLOR = 2
NO_OF_QUBITS_INDIVIDUAL = NO_OF_QUBITS_PER_COLOR*NO_OF_NODES
POPULATION_SIZE = 2**NO_OF_QUBITS_INDIVIDUAL #2*NO_OF_NODES#
NO_OF_MAX_GROVER_ITERATIONS = int(math.sqrt(2**NO_OF_QUBITS_FITNESS))

# Implementation

## Helper functions

* `to_binary` -- used for getting the two's complement representation.
* `pairs_colors` -- returns a list of tuples representing the color configuration for each node.
* `check_edges_validity` -- returns the number of edges between adjacent nodes colored using different colors.
* `get_number_of_edges` -- returns the number of edges in graph

In [7]:
def to_binary(value, number_of_bits, lsb=False):
    """
    Function return two's complement representation
    :param value: value in decimal representation
    :param number_of_bits: number of bits used for representation
    :returns: np.array that represents the binary representation
    >>> to_binary(10,4)
    array([1, 0, 1, 0])
    >>> to_binary(10,4,True)
    array([0, 1, 0, 1])
    """
    if lsb == True:
        return flip(array(list(binary_repr(value, number_of_bits)), dtype=int))
    return array(list(binary_repr(value, number_of_bits)), dtype=int)


In [8]:
def pairs_colors(colors_list):
    """
    Function returns a list of colors from the binary representation of the individual
    :param colors_list: binary representation of the individual
    :returns: list of pairs representing the color configuration for each node.
    """
    pairs=list()
    for i in range(0,len(colors_list),2):
        pairs.append((colors_list[i],colors_list[i+1]))
    return pairs






In [9]:
def check_edges_validity(graph, colors):
    """
    Function return the number of edges between adjacent nodes colored using different colors
    :param graph: adjacency matrix
    :param colors: list of colors
    :returns: number of edges between adjacent nodes colored using different colors
    """
    no_of_valid_edges = 0
    for color in colors:
        if color in INVALID_COLORS_LIST:
            return -1
    for i in range(NO_OF_NODES):
        for j in range(i + 1, NO_OF_NODES):
            if graph[i][j]:#daca am legatura
                if colors[j]==colors[i]:
                    continue
                else:
                    no_of_valid_edges +=1

    return no_of_valid_edges

In [10]:
def get_number_of_edges(graph):
    """
    Function return the number of edges in graph
    :param graph: adjacency matrix
    :returns: number of edges in graph
    """
    no_of_edges = 0
    for i in range(NO_OF_NODES):
        for j in range(i + 1, NO_OF_NODES):
            if graph[i][j]:
                no_of_edges +=1    
    return no_of_edges

## Subcircuits

### $U_{fit}$ subcircuit

`get_ufit_instruction` is the function that creates the $U_{fit}$ subcircuit. $U_{fit}$ is an unitary operator 
$U_{fit}:\ket{u}\otimes\ket{0}\rightarrow\ket{u}\otimes\ket{f_{fit}(u)}$, where $\ket{u}\otimes\ket{0}$ is the individual-fitness value quantum pair register.

After applying the 
$$
    U_{fit}:\sum_{i=0}^{2^N-1}\ket{u}_{i}\otimes\ket{0}_{i}\rightarrow\sum_{i=0}^{2^N-1}\ket{u}_{i}\otimes\ket{f_{fit}(u)}_{i}
$$
on the initial population we obtain an assessed population
$$
     \ket{\psi}_{2}=U_{fit}\ket{\psi}_{1}=\frac{1}{\sqrt{2^N}}\sum_{i=0}^{2^N-1}\ket{u}_{i}\otimes\ket{f_{fit}(u)}_{i}.
$$


In [11]:
def get_ufit_instruction():
    #define and initialize the individual quantum register
    ind_qreg = QuantumRegister(NO_OF_QUBITS_INDIVIDUAL,"ind_qreg")
    #define and initialize the fitness quantum register. 
    fit_qreg = QuantumRegister(NO_OF_QUBITS_FITNESS+1,"fit_qreg")
    #create the ufit subcircuit
    qc = QuantumCircuit(ind_qreg,fit_qreg,name="U$_fit$")
    for i in range(0,POPULATION_SIZE):
        """
        For each individual in population get the two's complement representation and 
        set the qubits on 1 using X-gate, according to the binary representation
        """
        individual_binary = to_binary(i, NO_OF_QUBITS_INDIVIDUAL, True)
        for k in range(0,NO_OF_QUBITS_INDIVIDUAL):
            if individual_binary[k] == 0:
                qc.x(ind_qreg[k])
        """
        Create list of colors from individual binary representation, calculate the fitness value
        and get the two's complement representation of the fitness value.
        """        
        colors = pairs_colors(individual_binary)
        #calculate valid score
        valid_score = check_edges_validity(GRAPH,colors)
        valid_score_binary = to_binary(valid_score,NO_OF_QUBITS_FITNESS,True)

        """
        Set the fitness value in fitness quantum register for each individual and mark it valid or invalid
        """
        for k in range(0,NO_OF_QUBITS_FITNESS):
            if valid_score_binary[k]==1:
                qc.mct([ind_qreg[j] for j in range(0,NO_OF_QUBITS_INDIVIDUAL)],fit_qreg[k])
        #if fitness value si greater than 0 then set the valid qubit to 1
        if valid_score > 0:
            qc.mct([ind_qreg[j] for j in range(0,NO_OF_QUBITS_INDIVIDUAL)],fit_qreg[NO_OF_QUBITS_FITNESS])
        #reset individual
        for k in range(0,NO_OF_QUBITS_INDIVIDUAL):
            if individual_binary[k] == 0:
                qc.x(ind_qreg[k])
        qc.barrier()
    return qc.to_instruction()

### Oracle subcircuit

The Oracle operates on the fitness register qubits except the validity qubit and uses two’s complement232number representation for marking the states. The Oracle is implemented using 2 two's complement **Quantum Ripple Carry Adders** (https://arxiv.org/abs/quant-ph/0410184). 

`get_oracle_instruction` creates the Oracle subcircuit. The parameter `positive_value_array` is the **max** value, 
$$max \in \mathbb{N}, max > 0, max \in [2^{M+1},2^{M+2}-1)$$,

from Reduced Quantum Genetic Algorithm.

In our instantiation, the **max** value is represented by the total number of edges in graph.

In [12]:
def get_oracle_instruction(positive_value_array):
    #define and initialize fitness quantum register
    fit_reg = QuantumRegister(NO_OF_QUBITS_FITNESS,"fqreg")
    #define and initialize max quantum register
    no_of_edges_reg=QuantumRegister(NO_OF_QUBITS_FITNESS,"noqreg")
    #define and initialize carry quantum register
    carry_reg = QuantumRegister(3,"cqreg")
    #define and initialize oracle workspace quantum register
    oracle = QuantumRegister(1,"oqreg")
    #create Oracle subcircuit
    oracle_circ = QuantumCircuit(fit_reg,no_of_edges_reg,carry_reg,oracle,name="O")
    
    #define majority operator
    def majority(circ,a,b,c):
        circ.cx(c,b)
        circ.cx(c,a)
        circ.ccx(a, b, c)
    #define unmajority operator
    def unmaj(circ,a,b,c):
        circ.ccx(a, b, c)
        circ.cx(c, a)
        circ.cx(a, b)
    #define the Quantum Ripple Carry Adder
    def adder_4_qubits(p, a0, a1, a2, a3, b0, b1, b2, b3, cin, cout):
        majority(p, cin, b0, a0)
        majority(p, a0, b1, a1)
        majority(p, a1, b2, a2)
        majority(p, a2, b3, a3)
        p.cx(a3, cout)
        unmaj(p, a2, b3, a3)
        unmaj(p, a1, b2, a2)
        unmaj(p, a0, b1, a1)
        unmaj(p, cin, b0, a0)
    
    """
    Subtract max value. We start by storing the max value in the quantum register. Such, considering that 
    all qubits are |0>, if on position i in positive_value_array there's 0, then qubit i will be negated. Otherwise, 
    if on position i in positive_value_array there's a 1, by default will remain 0 in no_of_edges_reg quantum
    register. For performing subtraction, carry-in will be set to 1.
    """
    for i in range(0,NO_OF_QUBITS_FITNESS):
        if positive_value_array[i]==0:
            oracle_circ.x(no_of_edges_reg[i])
    oracle_circ.x(carry_reg[0])

    adder_4_qubits(oracle_circ, no_of_edges_reg[0],no_of_edges_reg[1],no_of_edges_reg[2],no_of_edges_reg[3],
               fit_reg[0],fit_reg[1],fit_reg[2],fit_reg[3],
               carry_reg[0],carry_reg[1]);

    
    oracle_circ.barrier()
    """
    Reset the value in no_of_edges_reg and carry-in
    """
    oracle_circ.x(no_of_edges_reg)
    oracle_circ.x(carry_reg[0])
    
    """
    Mark the corresponding basis states by shifting their amplitudes.
    """
    
    oracle_circ.h(oracle[0])
    oracle_circ.mct([fit_reg[i] for i in range(0,NO_OF_QUBITS_FITNESS)],oracle[0])
    oracle_circ.h(oracle[0])
    
    """
    Restore the fitness value by adding max value.
    """
    adder_4_qubits(oracle_circ, no_of_edges_reg[0],no_of_edges_reg[1],no_of_edges_reg[2],no_of_edges_reg[3],
               fit_reg[0],fit_reg[1],fit_reg[2],fit_reg[3],
               carry_reg[0],carry_reg[2]);
    return oracle_circ.to_instruction()

### Grover diffuser subcircuit

`get_grover_iteration_subcircuit` creates the Grover diffuser subcircuit.

In [13]:
def get_grover_iteration_subcircuit():
    #define and initialize fitness quantum register
    fit_qreg = QuantumRegister(NO_OF_QUBITS_FITNESS+1,"fqreg")
    #define and initialize oracle workspace quantum register
    oracle_ws = QuantumRegister(1,"ows")
    #create grover diffuser subcircuit
    grover_circ = QuantumCircuit(fit_qreg,oracle_ws,name ="U$_s$")

    grover_circ.h(fit_qreg)
    grover_circ.x(fit_qreg)

    grover_circ.h(oracle_ws[0])

    grover_circ.mct(list(range(NO_OF_QUBITS_FITNESS+1)), oracle_ws[0])  # multi-controlled-toffoli

    grover_circ.h(oracle_ws[0])


    grover_circ.x(fit_qreg)
    grover_circ.h(fit_qreg)
    grover_circ.h(oracle_ws)

    return grover_circ.to_instruction()

## Putting it all together -> RQGA for graph coloring problem

* Initializes all individual-fitness register pairs $\ket{u}_{i} \otimes \ket{0}_{i}$ as 
$$
     \ket{\psi}_{1}=\frac{1}{\sqrt{2^N}}\sum_{i=0}^{2^N-1}\ket{u}_{i} \otimes \ket{0}_{i}.
$$
* Compute the unitary operation corresponding to fitness computation
$$
     \ket{\psi}_{2}=U_{fit}\ket{\psi}_{1}=\frac{1}{\sqrt{2^N}}\sum_{i=0}^{2^N-1}\ket{u}_{i}\otimes\ket{f_{fit}(u)}_{i}.
$$
* Apply Grover’s Algorithm:
    * Ask the oracle $\tilde{O}_{max}(f_{fit}(u))$ and if $\ket{f_{fit}(u)}_{i} \geqslant max$, then the corresponding $\ket{f_{fit}(u)}_{i}$ basis states are marked.
    * Apply Grover’s diffusion operator to augment the amplitude of the marked states, $\ket{\psi}_{i}=\ket{f_{fit}(u)}_{i}$ with $f_{fit}(u)\geqslant 0$, in the fitness register
* Measure $\ket{\psi}_{m-1}$ register to obtain the corresponding individual.

In [14]:
def run_algorithm(file):
    #Load IBMQ account
    IBMQ.load_account()
    #calculate the number of edges in graph
    pos_no_of_edges = get_number_of_edges(GRAPH)
    print("No of edges:{0}".format(pos_no_of_edges))
    #define a list for storing the results
    final_results = []
    
    #Start with one Grover iteration and run the algorithm with different number of iterations
    for iterations in range(4,NO_OF_MAX_GROVER_ITERATIONS+1):
        print("Running with {0} iterations".format(iterations))
        
        print("Preparing quantum registers and creating quantum circuit...")
        ind_qreg = QuantumRegister(NO_OF_QUBITS_INDIVIDUAL,"ireg")
        fit_qreg = QuantumRegister(NO_OF_QUBITS_FITNESS+1,"freg") #8 qubits fitness + 1 valid
        carry_qreg = QuantumRegister(9,"qcarry")
        oracle = QuantumRegister(1,"oracle")
        creg = ClassicalRegister(NO_OF_QUBITS_INDIVIDUAL,"reg")
        no_of_edges_qreg = QuantumRegister(NO_OF_QUBITS_FITNESS,"pos_max_qreg")
        
        print("Creating quantum circuit...")
        
        qc = QuantumCircuit(ind_qreg,fit_qreg,carry_qreg,oracle,no_of_edges_qreg,creg)

        print("Creating superposition of individuals...")
        qc.h(ind_qreg)
        qc.h(oracle)

        print("Getting maximum number of edges {0} binary representation...".format(pos_no_of_edges))
        pos_value_bin = to_binary(pos_no_of_edges,NO_OF_QUBITS_FITNESS,True)


        print("Getting ufit, oracle and grover iterations subcircuits...")
        ufit_instr = get_ufit_instruction()
        oracle_instr = get_oracle_instruction(pos_value_bin)
        grover_iter_inst = get_grover_iteration_subcircuit()

        print("Append Ufit instruction to circuit...")
        qc.append(ufit_instr, [ind_qreg[q] for q in range(0,NO_OF_QUBITS_INDIVIDUAL)]+
                              [fit_qreg[q] for q in range(0,NO_OF_QUBITS_FITNESS+1)]
                )
        
        for it in range(0,iterations):
            print("Append Oracle instruction to circuit...")
            
            qc.append(oracle_instr,[fit_qreg[q] for q in range(0,NO_OF_QUBITS_FITNESS)]+
                               [no_of_edges_qreg[q] for q in range(0,NO_OF_QUBITS_FITNESS)]+
                               [carry_qreg[0],carry_qreg[2*it+1],carry_qreg[2*it+2]]+
                               [oracle[0]])
            print("Append Grover Diffuser to circuit...")
            qc.append(grover_iter_inst, [fit_qreg[q] for q in range(0,NO_OF_QUBITS_FITNESS+1)]+[oracle[0]])
        
        print("Measure circuit...")
        qc.measure(ind_qreg,creg)
        
        simulation_results = []
        
        provider = IBMQ.get_provider(hub='ibm-q',group='open', project='main')
        backend = provider.get_backend('ibmq_qasm_simulator')
        shots = 1024
        print("Starting simulator...")
        mapped_circuit = transpile(qc, backend=backend)
        qobj = assemble(mapped_circuit, backend=backend, shots=shots)
        # Perform 10 measurements for each circuit
        for run in range(0,10000):
            print("Run {0}\n".format(run))
            file.write("Run {0}\n".format(run))
            #print("Setup simulator...")    
            #shots = 8192
            try:
                runner = backend.run(qobj)
                job_monitor(runner)
                results = runner.result()
                answer = results.get_counts()
                #Get the result with the maximum number of counts
                max_item =max(answer.items(), key=operator.itemgetter(1))
                print(max_item[0])
                #Store the result.
                
                file.write("{0}".format(answer))
                file.write("\n")
                simulation_results.append(max_item)
            except Exception as e:
                print(str(e))
                print("Error on run {0} with {1} grover iterations".format(run,iterations))
        print("\n-------------------------\n")
        final_results.append((iterations,simulation_results))
    file.write("RESULTS:\n")
    print(final_results)
    file.write("{0}".format(final_results))

In [15]:
import sys

In [16]:
with open('results.txt', 'w') as f:
    run_algorithm(f)

No of edges:5
Running with 4 iterations
Preparing quantum registers and creating quantum circuit...
Creating quantum circuit...
Creating superposition of individuals...
Getting maximum number of edges 5 binary representation...
Getting ufit, oracle and grover iterations subcircuits...
Append Ufit instruction to circuit...
Append Oracle instruction to circuit...
Append Grover Diffuser to circuit...
Append Oracle instruction to circuit...
Append Grover Diffuser to circuit...
Append Oracle instruction to circuit...
Append Grover Diffuser to circuit...
Append Oracle instruction to circuit...
Append Grover Diffuser to circuit...
Measure circuit...
Starting simulator...
Run 0



D:\Soft\anaconda\envs\quantum\lib\site-packages\qiskit\providers\ibmq\ibmqbackend.py:906: DeprecationWarning: Passing a Qobj to Backend.run is deprecated and will be removed in a future release. Please pass in circuits or pulse schedules instead.
  return super().run(circuits, job_name=job_name,


Job Status: job has successfully run
1000111101
Run 1

Job Status: job has successfully run
1001110100
Run 2

Job Status: job has successfully run
1000110111
Run 3

Job Status: job has successfully run
1001110100
Run 4

Job Status: job has successfully run
0100110010
Run 5

Job Status: job has successfully run
0101101110
Run 6

Job Status: job has successfully run
1001001111
Run 7

Job Status: job has successfully run
1000001100
Run 8

Job Status: job has successfully run
0010101000
Run 9

Job Status: job has successfully run
1011100000
Run 10

Job Status: job has successfully run
0101110111
Run 11

Job Status: job has successfully run
1100111101
Run 12

Job Status: job has successfully run
1001110011
Run 13

Job Status: job has successfully run
0100111011
Run 14

Job Status: job has successfully run
0111100100
Run 15

Job Status: job has successfully run
1011000000
Run 16

Job Status: job has successfully run
0110101011
Run 17

Job Status: job has successfully run
0111100000
Run 18

J

1001100010
Run 146

Job Status: job has successfully run
0000010001
Run 147

Job Status: job has successfully run
0011011010
Run 148

Job Status: job has successfully run
1110101000
Run 149

Job Status: job has successfully run
1001111010
Run 150

Job Status: job has successfully run
1110000110
Run 151

Job Status: job has successfully run
1011111001
Run 152

Job Status: job has successfully run
0111101010
Run 153

Job Status: job has successfully run
0101000011
Run 154

Job Status: job has successfully run
1001010111
Run 155

Job Status: job has successfully run
1011101101
Run 156

Job Status: job has successfully run
1011111000
Run 157

Job Status: job has successfully run
1111101010
Run 158

Job Status: job has successfully run
1011111101
Run 159

Job Status: job has successfully run
1001001110
Run 160

Job Status: job has successfully run
1010111110
Run 161

Job Status: job has successfully run
1010100011
Run 162

Job Status: job has successfully run
1000110111
Run 163

Job Status:

KeyboardInterrupt: 